# 1. Import thư viện và nhập dữ liệu từ file

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import emoji # For checking trash comments

In [2]:
FANPAGE_LINK = 'unity3d'
FOLDER_PATH = 'Data/'

In [3]:
pd.set_option('display.max_colwidth',None) # Dùng để hiển thị toàn bộ văn bản trong pandas nếu quá dài
extracted_df = pd.read_csv('Data/unity3dExtracted.csv')

In [4]:
extracted_df.info()
#extracted_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_id         220 non-null    int64 
 1   post_text       218 non-null    object
 2   time            220 non-null    object
 3   reaction_count  220 non-null    int64 
 4   reactions       219 non-null    object
 5   shares          220 non-null    int64 
 6   comments        220 non-null    int64 
 7   comments_full   220 non-null    object
dtypes: int64(4), object(4)
memory usage: 13.9+ KB


# 2. Làm sạch dữ liệu

## <span style = "font-size: smaller"> 2.1 Cấu trúc của dữ liệu</span> 

Dữ liệu được lấy về có dạng
| Tên biến           | Ý nghĩa                            | Dạng dữ liệu |  Miền dữ liệu | Chú thích |
|--------------------|------------------------------------|--------------|---------------|-----------|
|post_id             |Id của bài viết                     |str           |               |           |  
|post_text           |Nội dung bài viết                   |str           |               |           |    
|time                |Thời gian bài viết được đăng        |str(datetime) |               |           |      
|reaction_count      |Số lượt reaction                    |int           | >= 0          |           |
|reactions           |Số lượt reaction từng loại          |str(dict)     |               |Chú thích 1|
|shares              |Số lượt chia sẻ của bài viết        |int           | >= 0          |           |
|comments            |Số lượt bình luận                   |int           | >= 0          |           |
|comments_full       |Nội dung của bình luận              |str(list)     |               |Chú thích 2|

-Chú thích 0:  
&nbsp;&nbsp;&nbsp;&nbsp;str(datatype) là dữ liệu lưu ở dạng str tuy nhiên lưu theo syntax của datatype và có thể parse sang datatype   
&nbsp;&nbsp;&nbsp;&nbsp;Id của bài viết có thể dùng để truy cập theo đường link dạng https://www.facebook.com/id  
-Chú thích 1: Cấu trúc của 1 reactions:  
&nbsp;&nbsp;&nbsp;&nbsp;{'like': int, 'love': int, 'haha':int, 'wow': int, 'care': int,'sad': int,'angry': int} với int > 0  
&nbsp;&nbsp;&nbsp;&nbsp;Nếu int = 0 thì sẽ không hiển thị , ví dụ với 1 reactions chỉ có like = 1, còn lại = 0 sẽ có dạng {'like': 1}  
-Chú thích 2: Cấu trúc của 1 comments:  
&nbsp;&nbsp;&nbsp;&nbsp;-Chứa danh sách cách comment dạng: \[comment1,comment2,comment3,...,commentN\]  
&nbsp;&nbsp;&nbsp;&nbsp;-Với mỗi comment, có cấu trúc dạng từ điển (dict) giống với reactions ở chú thích 1 với các trường dữ liệu sau:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;comment_id, comment_url, commenter_id, commenter_url, commenter_name, commenter_meta, comment_text  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;comment_time, comment_image, comment_reactors, comment_reactions, comment_reaction_count, replies  
&nbsp;&nbsp;&nbsp;&nbsp;-Đặc biệt đối với replies có cấu trúc dạng list: \[reply1,reply2,reply3,...,replyN\]  
&nbsp;&nbsp;&nbsp;&nbsp;-Với mỗi replies có cấu trúc dạng từ điển (dict) với các trường dữ liệu sau:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;comment_id, comment_url, commenter_id, commenter_url, commenter_name, commenter_meta, comment_text  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;comment_time, comment_image, comment_reactors, comment_reactions, comment_reaction_count  
&nbsp;&nbsp;&nbsp;&nbsp;-Phản hổi bình luận của phản hồi bình luận(replies của replies) được tính chung là phản hồi của bình luận(replies), do đó nên phản hồi bình luận (replies) không có trường dữ liệu replies

Dữ liệu sau khi được xử lý có dạng
| Tên biến           | Ý nghĩa                            | Dạng dữ liệu |  Miền dữ liệu | Chú thích |
|--------------------|------------------------------------|--------------|---------------|-----------|
|Id                  |Id của bài viết                     |str           |               |           | 
|Text                |Nội dung bài viết                   |str           |               |           |    
|Time                |Thời gian bài viết được đăng        |datetime(str) |               |Chú thích 0|      
|Reaction Count      |Số lượt reaction                    |int           | >= 0          |           |
|Share Count         |Số lượt chia sẻ của bài viết        |int           | >= 0          |           |  
|Comment Count       |Số lượt bình luận                   |int           | >= 0          |           |
|Comments            |Nội dung của bình luận              |list(str)     |               |Chú thích 1|
|Like Count          |Số lượt thích của bài viết          |int           | >= 0          |           |
|Love Count          |Số lượt yêu thích của bài viết      |int           | >= 0          |           |
|Haha Count          |Số lượt haha của bài viết           |int           | >= 0          |           |
|Wow Count           |Số lượt wow của bài viết            |int           | >= 0          |           |
|Care Count          |Số lượt thương thương của bài viết  |int           | >= 0          |           |
|Sad Count           |Số lượt buồn của bài viết           |int           | >= 0          |           |
|Angry Count         |Số lượt phẫn nộ của bài viết        |int           | >= 0          |           |  

Chú thích 0 :  
&nbsp;&nbsp;&nbsp;&nbsp;-Dữ liệu thể hiện ở dạng datetime nhưng khi lưu thành file csv sẽ ở dạng str
Chú thích 1 : Cấu trúc của 1 Comments:  
&nbsp;&nbsp;&nbsp;&nbsp;-Chứa danh sách cách comment dạng: \[comment1,comment2,comment3,...,commentN\]  
&nbsp;&nbsp;&nbsp;&nbsp;-Với mỗi comment, có cấu trúc dạng từ điển (dict) với các trường dữ liệu sau:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Name, Text, Reaction Count, Reactions, Time  
&nbsp;&nbsp;&nbsp;&nbsp;-Phần phản hồi bình luận (replies) được tách ra thành bình luận riêng  
&nbsp;&nbsp;&nbsp;&nbsp;-Dữ liệu thể hiện ở dạng list nhưng khi lưu thành file csv sẽ ở dạng str

## <span style = "font-size: smaller"> 2.2 Một số hàm phụ</span> 

In [5]:
commentsExtracted = False
reactionTypes = ['like','love','haha','wow','care','sad','angry']

In [6]:
# Trích xuất các số từ xâu (str)
def ExtractNumber(inputStr : str = None) -> list:
    if (type(inputStr) != str):
        return []
    result = []
    currNumber = ''
    for c in inputStr:
        if (c.isdigit()):
            currNumber += c
        elif (currNumber != ''):
            result.append(int(currNumber))
            currNumber = ''
    if (currNumber != ''):
        result.append(int(currNumber))
    return result

In [7]:
# Thay thế datetime.datetime(data) bằng 'data'
def DateTimeReplace(inputStr : str = None):
    while ('datetime.datetime' in inputStr):
        startIndex = inputStr.find('datetime.datetime')
        endIndex = startIndex
        for endIndex in range(startIndex,len(inputStr)):
            if (inputStr[endIndex-1:endIndex+1] == "),"):
                break
        inputStr = inputStr.replace(inputStr[startIndex:endIndex],str(ExtractNumber(inputStr[startIndex:endIndex])).replace("[","'").replace("]","'"))
    return inputStr
# Hàm trích xuất comment cũ, giữ nguyên cấu trúc như ban đầu
def ExtractCommentOriginal(inputDict : dict = None,keys : list = None,extractReplies = True,repliesKey : str = 'replies',idKey : str = 'comment_id',fullReaction : bool = False) -> dict:
    extractedDict = {}
    for key in keys:
        try:
            extractedDict[key[1]] = inputDict[key[0]]
        except:
            extractedDict[key[1]] = str(inputDict[key[0]])
        try:
            defaultValue = key[2]
            extractedDict[key[1]] = defaultValue
        except:
            pass
    if (extractReplies):
        numReplies = len(inputDict[repliesKey])
        nestedSeries = {}
        for i in range(numReplies):
            nestedSeries[str(i)] = ExtractCommentOriginal(inputDict[repliesKey][i],keys,extractReplies=False)
        extractedDict['Replies'] = nestedSeries
        if (fullReaction):
            for reactType in reactionTypes:
                try:
                    if (reactType not in extractedDict['Reactions']):
                        extractedDict['Reactions'][reactType] = 0
                except:
                    extractedDict['Reactions'] = {reactType : 0}
    return extractedDict
# Hàm trích xuất comment
def ExtractComment(inputList : dict = None,keys : list = None,idKey : str = 'comment_id',repliesKey : str = 'replies'
                   ,fullReaction : bool = False) -> list:
    resultList = []
    def ExtractData(resultList : list = None,inputDict : dict = None,keys : list = None) -> None:
        resultDict = {}
        for key in keys:
            try:
                resultDict[key[1]] = inputDict[key[0]]
            except:
                resultDict[key[1]] = str(inputDict[key[0]])
            try:
                defaultValue = key[2]
                resultDict[key[1]] = defaultValue
            except:
                pass
        resultList.append(resultDict)
    for currComment in inputList:
        ExtractData(resultList=resultList,inputDict=currComment,keys=keys)
        if (fullReaction):
            numReplies = len(currComment[repliesKey])
            for i in range(numReplies):
                ExtractData(resultList=resultList,inputDict=currComment[repliesKey][i],keys=keys)
    return resultList


## <span style = "font-size: smaller"> 2.3 Xử lý dữ liệu lỗi</span> 

### <span style = "font-size: smaller"> Các trường hợp dữ liệu bị lỗi:</span>
    1-Số lượt reaction_count không bằng tổng của từng loại reactions -> xóa bỏ
    2-Số lượt reaction_count bằng không -> báo lỗi
    3-Số lượt comment khác 0 nhưng comment bị rỗng -> xóa bỏ
    4-Số lượt comment bằng 0 -> báo lỗi
    5-Số lượt shares bằng 0 -> báo lỗi
    6-Nội dung bài viết trống -> xóa bỏ (đây là các bài Reels, chỉ có video)

In [8]:

length = len(extracted_df)
errorRow = set()
warningRow = set()
mismatchReactCount = 0
for i in range(length):
    # Đánh dấu trường hợp 1
    if (extracted_df['reaction_count'].iloc[i] != sum(ExtractNumber(extracted_df['reactions'].iloc[i]))):
        errorRow.add(i)
        mismatchReactCount += 1
    # Đánh dấu trường hợp 2
    if (extracted_df['reaction_count'].iloc[i] == 0):
        print('0 Reaction at ' + str(i))
        warningRow.add(i)
mismatchCommentCount = 0
for i in range(length):
    # Đánh dấu trường hợp 3
    if (extracted_df['comments_full'].iloc[i] == '[]' and extracted_df['comments'].iloc[i] != 0 ):
        errorRow.add(i)
        mismatchCommentCount += 1
    # Đánh dấu trường hợp 4
    if (extracted_df['comments'].iloc[i]  == 0):
        print('0 Comment at ' + str(i))
        warningRow.add(i)
for i in range(length):
    # Đánh dấu trường hợp 5
    if (extracted_df['shares'].iloc[i] == 0):
        print('0 Share at ' + str(i))
        warningRow.add(i)
for i in range(length):
    # Đánh dấu trường hợp 6
    if (pd.isna(extracted_df['post_text'].iloc[i])):
        errorRow.add(i)
    else:
        # Xóa bỏ kí tự xuống dòng để lưu file csv đẹp hơn
        extracted_df['post_text'].at[i] = extracted_df['post_text'].iloc[i].replace('\n' ,'') 
# Thông báo lỗi và cảnh báo
if (mismatchReactCount != 0):
    print(str(mismatchReactCount) + " mismatch react count")
if (mismatchCommentCount != 0):
    print(str(mismatchCommentCount) + " mismatch comment count")
print(errorRow)
print(warningRow)


0 Comment at 60
0 Comment at 64
0 Comment at 74
0 Comment at 79
0 Comment at 114
0 Comment at 119
0 Comment at 125
0 Comment at 126
0 Comment at 129
0 Comment at 140
0 Comment at 149
0 Comment at 151
0 Comment at 157
0 Comment at 162
0 Comment at 169
0 Comment at 171
0 Comment at 172
0 Comment at 173
0 Comment at 174
0 Comment at 175
0 Comment at 176
0 Comment at 187
0 Comment at 188
0 Comment at 199
0 Comment at 205
0 Comment at 208
0 Comment at 210
0 Comment at 215
0 Comment at 217
0 Share at 27
0 Share at 44
0 Share at 124
0 Share at 129
0 Share at 151
0 Share at 172
0 Share at 173
0 Share at 199
0 Share at 201
0 Share at 208
0 Share at 210
0 Share at 217
0 Share at 218
1 mismatch react count
27 mismatch comment count
{11, 142, 145, 165, 166, 37, 170, 44, 179, 180, 184, 191, 193, 67, 196, 197, 198, 200, 204, 86, 218, 96, 97, 103, 105, 109, 123, 124}
{129, 140, 149, 151, 27, 157, 162, 169, 171, 172, 173, 174, 175, 176, 44, 187, 60, 188, 64, 199, 201, 74, 205, 79, 208, 210, 215, 217, 

### <span style = "font-size: smaller"> Tạo dataframe mới chỉ bao gồm các dòng hợp lệ </span>

In [9]:
df = pd.DataFrame(columns=extracted_df.columns)
for i in range(length):
    if (i in errorRow):
        continue
    df.loc[len(df)] = extracted_df.iloc[i]
length = len(df)

In [10]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 0 to 191
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_id         192 non-null    object
 1   post_text       192 non-null    object
 2   time            192 non-null    object
 3   reaction_count  192 non-null    object
 4   reactions       192 non-null    object
 5   shares          192 non-null    object
 6   comments        192 non-null    object
 7   comments_full   192 non-null    object
dtypes: object(8)
memory usage: 13.5+ KB


,post_id,post_text,time,reaction_count,reactions,shares,comments,comments_full
count,192,192,192,192,192,192,192,192
unique,192,192,192,136,187,44,40,164
top,722635819896061,Exciting news for student creators! 🌟☁️ Early access to Unity Cloud🎟️ 20% stackable discount in the Asset Store⚙️ Version Control to level up your projectsFind out more about the Unity Student Plan's new perks: on.unity.com/3SNYdAd,2023-11-24 00:00:02,29,"{'like': 35, 'love': 3}",4,0,[]
freq,1,1,1,5,2,15,29,29


# 3. Tiền xử lý dữ liệu

## <span style = "font-size: smaller"> 3.1 Xử lý lượt reaction từng loại</span>

Tách reactions ra thành 7 cột riêng biệt và xóa cột reactions cũ

In [11]:
for i in range(length):
    try:
        df['reactions'].at[i] = ast.literal_eval(df['reactions'].iloc[i])
    except:
        df['reactions'].at[i] = {}
for reactType in reactionTypes:
    df[reactType] = None
for i in range(length):
    total = 0
    for reactType in reactionTypes:
        if (reactType in df['reactions'].iloc[i]):
            df[reactType].at[i] = df['reactions'].iloc[i][reactType]
            total += int(df[reactType].iloc[i])
        else:
            df[reactType].at[i] = 0
    df['reaction_count'].at[i] = total
df = df.drop(columns='reactions')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 0 to 191
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_id         192 non-null    object
 1   post_text       192 non-null    object
 2   time            192 non-null    object
 3   reaction_count  192 non-null    object
 4   shares          192 non-null    object
 5   comments        192 non-null    object
 6   comments_full   192 non-null    object
 7   like            192 non-null    object
 8   love            192 non-null    object
 9   haha            192 non-null    object
 10  wow             192 non-null    object
 11  care            192 non-null    object
 12  sad             192 non-null    object
 13  angry           192 non-null    object
dtypes: object(14)
memory usage: 26.6+ KB


## <span style = "font-size: smaller"> 3.2 Xử lý bình luận </span>

### <span style = "font-size: smaller"> Dựng lại cấu trúc của comments</span>

In [12]:
# Các trường dữ liệu cần trích xuất ['tên biến ban đầu','tên biến sau khi trích xuất','giá trị mặc định']
fieldToExtract = [['commenter_name','Name'],['comment_text','Text'],['comment_reaction_count','Reaction Count',0]
                  ,['comment_time','Time']]
flag = False
for i in range(length):
    raw_comment = df['comments_full'].iloc[i]
    raw_comment = DateTimeReplace(raw_comment) # Xử lý datetime.datetime() để sử dụng ast.literal_eval, nếu không sẽ bị lỗi parse
    try:
        list_comments = ast.literal_eval(raw_comment) # Parse comment từ str sang list/dict
        df['comments_full'].at[i] = ExtractComment(inputList=list_comments,keys=fieldToExtract) # Trích xuất dữ liệu
    except:
        flag = True
        break
# Xử lý nếu như chạy khu vực lệnh này nhiều hơn 1 lần hoặc quá trình chạy bị lỗi
if (not flag):
    commentsExtracted = True
    print("Success")
else:
    if (commentsExtracted):
        print("Comment Already Extracted")
    else:
        print("Error")
    

Success


### <span style = "font-size: smaller"> Xóa bình luận rác</span>
Các trường hợp bình luận rác:  
&nbsp;&nbsp;&nbsp;&nbsp;1-Có chứa 4 emoji (trên 3) và chứa link http

In [13]:
def RemoveTrashComment(comment : list = None,cmtCheckDefs : list = None) -> [list,int]:
    totalRemoved = 0
    if (type(comment) != None and cmtCheckDefs):
        cmtLen = len(comment)
        index = 0
        while(index < cmtLen):
            for cmtCheck in cmtCheckDefs:
                if (cmtCheck(comment[index]['Text'])):
                    totalRemoved += 1
                    comment.remove(comment[index])
                    index -= 1
                    cmtLen = len(comment)
                    break
            index += 1
        return [comment,totalRemoved]
    elif (comment):
        return [comment,totalRemoved]
    else:
        return [[],totalRemoved]
def TrashCommentCheckerType1(inputStr : str = None) -> bool: # Xử lý trường hợp 1
    keyWords = 'http'
    threshold = 1
    for keyWord in keyWords:
        if (keyWord in inputStr and emoji.emoji_count(inputStr) >= threshold):
            return(True)
    return False

In [14]:
checkerDefs = [TrashCommentCheckerType1] # Các hàm xét bình luận rác
totalRemoved = 0
for i in range(length):
    cmt = RemoveTrashComment(df['comments_full'].iloc[i],checkerDefs)
    df['comments_full'].at[i] = cmt[0]
    totalRemoved += cmt[1]
print(totalRemoved)

89


# 4. Xuất ra file csv

In [15]:
columnsName = {'post_id':'Id','post_text':'Text','time':'Time','reaction_count':'Reaction Count'
               ,'shares':'Share Count','comments':'Comment Count','comments_full':'Comments'
               ,'like':'Like Count','love':'Love Count','haha':'Haha Count','sad':'Sad Count','angry':'Angry Count'
               ,'wow':'Wow Count','care':'Care Count'}
df = df.rename(columns=columnsName)

In [16]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 0 to 191
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id              192 non-null    object
 1   Text            192 non-null    object
 2   Time            192 non-null    object
 3   Reaction Count  192 non-null    object
 4   Share Count     192 non-null    object
 5   Comment Count   192 non-null    object
 6   Comments        192 non-null    object
 7   Like Count      192 non-null    object
 8   Love Count      192 non-null    object
 9   Haha Count      192 non-null    object
 10  Wow Count       192 non-null    object
 11  Care Count      192 non-null    object
 12  Sad Count       192 non-null    object
 13  Angry Count     192 non-null    object
dtypes: object(14)
memory usage: 26.6+ KB


,Id,Text,Time,Reaction Count,Share Count,Comment Count,Comments,Like Count,Love Count,Haha Count,Wow Count,Care Count,Sad Count,Angry Count
count,192,192,192,192,192,192,192,192,192,192,192,192,192,192
unique,192,192,192,136,44,40,159,130,59,15,18,11,4,27
top,722635819896061,Exciting news for student creators! 🌟☁️ Early access to Unity Cloud🎟️ 20% stackable discount in the Asset Store⚙️ Version Control to level up your projectsFind out more about the Unity Student Plan's new perks: on.unity.com/3SNYdAd,2023-11-24 00:00:02,29,4,0,[],29,1,0,0,0,0,0
freq,1,1,1,5,15,29,33,6,17,138,100,101,171,131


In [17]:
FILENAME = 'unity3dCleaned'
FOLDER_PATH = 'Data/'
path=FOLDER_PATH + FILENAME + ".csv"
df.to_csv(path, index=False)

# 5. Các hàm hỗ trợ quan sát dữ liệu

In [18]:
commentCountDict = {}
for i in range(length):
    try:
        for j in range(len(df['Comments'].iloc[i])):
            try:
                commentCountDict[df['Comments'].iloc[i][j]['Text']] += 1
            except:
                commentCountDict[df['Comments'].iloc[i][j]['Text']] = 1
    except:
        print('ERORR at ' + str(i))
        pass
commentCountList = []
for ele in commentCountDict:
    commentCountList.append([commentCountDict[ele],ele])
commentCountList.sort(reverse=True)
for ele in commentCountList:
    print(ele)

[7, 'Braden Richard Dozler']
[6, 'Nice']
[5, 'Madhav Bomma']
[5, 'Keep up the good work!']
[4, 'Gaza']
[3, '👍']
[3, 'nice']
[3, 'https://\nzap.struckd.com/\ngames/\nluks-araba-oyunu\n-1']
[3, 'Thank you for sharing!']
[3, 'Godot is the future. Donate to GODOT, they have made huge improvements as an open source game engine. https://\ngodotengine.org/']
[3, 'Elijah Ishmael']
[3, 'Be careful with this company. I signed up for their free subscription and I canceled it right the same day I was going to get subscribed they charged me 555.00 for a service I didn’t use and now they don’t want to refund. They don’t reply to my emails . Be aware and don’t fall like I did.']
[2, 'wow']
[2, 'Unity Why is there no support for localizing the program lists? Please add Arabic language support in the program']
[2, 'Thank you for sharing this!']
[2, 'Steve Green']
[2, 'Samantha Jacobs']
[2, 'Robert Mccain']
[2, 'Rafik Benamaraa']
[2, 'Ophir Klainman']
[2, 'Nice 👍']
[2, 'Love it']
[2, "I'm interested"]
[